In [1]:
import pandas as pd
import json
import os

Evaluation setup:  
* Embedding model - Text Embedding 004 (Google)  
* LLM - Gemini 2.5 Flash  


## RAGAS Answer Correctness:

The assessment of Answer Correctness involves measuring the accuracy of the generated answer when compared to the ground truth. This evaluation relies on the ground truth and the answer, with scores ranging from 0 to 1. A higher score indicates a closer alignment between the generated answer and the ground truth, signifying better correctness.
Answer correctness  is computed as the sum of factual correctness and the semantic similarity between the given answer and the ground truth.  

Factual correctness is a metric that compares and evaluates the factual accuracy of the generated response with the reference. This metric is used to determine the extent to which the generated response aligns with the reference. The factual correctness score ranges from 0 to 1, with higher values indicating better performance. To measure the alignment between the response and the reference, the metric uses the LLM to first break down the response and reference into claims and then uses natural language inference to determine the factual overlap between the response and the reference. Factual overlap is quantified using precision, recall, and F1 score, which can be controlled using the mode parameter. By default, the mode is set to F1, you can change the mode to precision or recall by setting the mode parameter.

Answer similarity is calculated by following steps:  
Step 1: Vectorize the ground truth answer using the embedding model.  
Step 2: Vectorize the generated answer using the same embedding model.  
Step 3: Compute the cosine similarity between the two vectors.  
        
By default "text-embedding-ada-002" model is used. In that evaluation, we used Text Embedding 004 (Google).  

Final score is created by taking a weighted average of the factual correctness (F1 score) and the semantic similarity. 
(By default, there is a 0.75 : 0.25 weighting.)   

Total API Calls: 4  
* 1 LLM call to produce the "simple statements  
* 1 LLM call to determine the true positives, false positives, and false negatives  
* 1 embedding call to embed the context  
* 1 embedding call to embed the AI answer  

I feel like RAGAS use much more in terms of LLM calls. This is a source code:  
https://github.com/explodinggradients/ragas/blob/main/ragas/src/ragas/metrics/_answer_correctness.py

There is a compaint about RAGAS costing 700$ for 100 QA set:  
https://www.reddit.com/r/LangChain/comments/1dbmqii/i_spent_700_on_evaluating_100_rag_qa_set_using/

Because the generated results is non-deterministic, I run evaluation 3 times and use mean number. 370 questions x 3 times x 2 (vanilla and RAG) X 4 API calls = 8880 API calls total for one set of results.
When I decided to play with Gemini 2.5 flash, ​I also got results for another metrics, in total dashboard shows 16000 API calls and 167 dollars spent.  
Conclusion: I will stick with Gemini 2.0 flash model as main judge for the rest of the project

Source:  
https://docs.ragas.io/en/v0.1.21/concepts/metrics/answer_correctness.html  
https://github.com/dkhundley/llm-rag-guide/blob/main/notebooks/ragas.ipynb  
https://docs.ragas.io/en/stable/references/embeddings/#ragas.embeddings.embedding_factory  
https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/factual_correctness/#factual-correctness  
https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/semantic_similarity/  

In [2]:
file_name = "test_dataset_together_meta-llama_Llama-4-Scout-17B-16E-Instruct_top5_answered.json"
base_name = file_name.replace('.json', '')

    # Use forward slashes or os.path.join for better cross-platform compatibility
VANILLA_ANSWER_CORRECTNESS = f"{base_name}_vanilla_answer_correctness_evaluated.json"
RAG_ANSWER_CORRECTNESS = f"{base_name}_rag_answer_correctness_evaluated.json"
RAG_ANSWER_SIMILARITY = f"{base_name}_rag_answer_similarity_evaluated.json"
VANILLA_ANSWER_SIMILARITY = f"{base_name}_vanilla_answer_similarity_evaluated.json"
RAG_ANSWER_RELEVANCY = f"{base_name}_rag_answer_relevancy_evaluated.json"
RAG_FAITHFULNESS = f"{base_name}_rag_faithfulness_evaluated.json"

with open(VANILLA_ANSWER_CORRECTNESS, 'r', encoding='utf-8') as f:
    data = json.load(f)   
df_1 = pd.DataFrame(data)

with open(RAG_ANSWER_CORRECTNESS, 'r', encoding='utf-8') as f:
    data = json.load(f) 
df_2 = pd.DataFrame(data)
df_2 = df_2[["Modified Questions", 
                 "Answer Correctness for RAG run 1", 
                 "Answer Correctness for RAG run 2",
                 "Answer Correctness for RAG run 3",
                 "Mean Answer Correctness for RAG"]]
    
merged_df = pd.merge(df_1, df_2, on="Modified Questions", how="inner")
# Calculate overall means
vanilla_mean = merged_df['Mean Answer Correctness for vanilla'].mean()
rag_mean = merged_df['Mean Answer Correctness for RAG'].mean()
    
print(f"Overall Mean Answer Correctness for Vanilla: {vanilla_mean:.4f}")
print(f"Overall Mean Answer Correctness for RAG: {rag_mean:.4f}")
print(f"Difference (RAG - Vanilla): {rag_mean - vanilla_mean:.4f}")   

Overall Mean Answer Correctness for Vanilla: 0.6058
Overall Mean Answer Correctness for RAG: 0.5695
Difference (RAG - Vanilla): -0.0364


Again, vanilla LLM response is evaluated higher than RAG-enhanced. 

## RAGAS Answer semantic similarity  

The metric is a part of RAGAS Answer Correctness metric.  

The Answer Correctness's final score is created by taking a weighted average of the factual correctness (F1 score) and the semantic similarity. 
(By default, there is a 0.75 : 0.25 weighting.) 

This evaluation is based on the ground truth and the answer, with values falling within the range of 0 to 1. A higher score signifies a better alignment 
between the generated answer and the ground truth.  
Step 1: Vectorize the ground truth answer using the specified embedding model.  
Step 2: Vectorize the generated answer using the same embedding model.  
Step 3: Compute the cosine similarity between the two vectors.  

https://docs.ragas.io/en/v0.1.21/concepts/metrics/semantic_similarity.html
https://github.com/dkhundley/llm-rag-guide/blob/main/notebooks/ragas.ipynb

Total API Calls: 2
* 1 embedding call to embed the ground truth
* 1 embedding call to embed the AI answer  

The metric is produced much faster (around 30 minutes for the whole set of results). It is 25% component of Answer Correctness metric.   



In [ ]:

# SET OF METRIC FOR VANILLA ISN'T COMPELETED, SO I WILL SKIP IT FOR NOW

with open(RAG_ANSWER_SIMILARITY, 'r', encoding='utf-8') as f:
    data = json.load(f) 
df_3 = pd.DataFrame(data)
df_3 = df_3[["Modified Questions", 
            "Answer Semantic Similarity for rag"]]


with open(VANILLA_ANSWER_SIMILARITY, 'r', encoding='utf-8') as f:
    data = json.load(f) 
df_4 = pd.DataFrame(data)
df_4 = df_4[["Modified Questions", 
            "Answer Semantic Similarity for vanilla"]]

merged_df = pd.merge(merged_df, df_3, on="Modified Questions", how="inner")
merged_df = pd.merge(merged_df, df_4, on="Modified Questions", how="inner")

# Calculate overall means
rag_similarity_mean = merged_df['Answer Semantic Similarity for rag'].mean()
vanilla_similarity_mean = merged_df['Answer Semantic Similarity for vanilla'].mean()
print(f"Overall Mean Answer Semantic Similarity for RAG: {rag_similarity_mean:.4f}")
print(f"Overall Mean Answer Semantic Similarity for Vanilla: {vanilla_similarity_mean:.4f}")
print(f"Difference (RAG - Vanilla): {rag_similarity_mean - vanilla_similarity_mean:.4f}")

# Group by psychiatric category
category_stats = merged_df.groupby('psychiatric_category').agg({
        'Answer Semantic Similarity for vanilla': 'mean',
        'Answer Semantic Similarity for rag': 'mean'}).round(4)
    
    # Flatten column names
category_stats.columns = ['Vanilla_Mean', 'RAG_Mean']
    
    # Add difference column
category_stats['Difference (RAG - Vanilla)'] = (category_stats['RAG_Mean'] - 
                                                   category_stats['Vanilla_Mean']).round(4)
    
    # Sort by difference to see which categories benefit most from RAG
category_stats = category_stats.sort_values('Difference (RAG - Vanilla)', ascending=False)
    
print(category_stats)

Overall Mean Answer Semantic Similarity for RAG: 0.8752
Overall Mean Answer Semantic Similarity for Vanilla: 0.8797
Difference (RAG - Vanilla): -0.0044
                                                    Vanilla_Mean  RAG_Mean  \
psychiatric_category                                                         
Bipolar Disorders                                         0.8346    0.8783   
Somatic Disorders                                         0.8396    0.8762   
Other Mental Disorders                                    0.8819    0.8709   
Depressive Disorders                                      0.9060    0.8630   
Anxiety Disorders                                         0.9342    0.8850   
Dissociative Disorders                                       NaN    0.9323   
Eating Disorders                                             NaN    0.8656   
Obsessive-Compulsive Disorders                               NaN    0.8811   
Personality Disorders                                        NaN    

# RAG Metrics  
## RAGAS Faithfulness 
Faithfulness measures the factual consistency of the generated answer against the given context. It is calculated from the answer and the retrieved context. The answer is scaled to the (0, 1) range. The higher the better. 
The generated answer is regarded as faithful if all the claims made in the answer can be inferred from the given context.  
At the first step, the generated answer is broken down into individual statements.     
Then each of these claims is cross-checked with the given context to determine if it can be inferred from the context.    
The final score is calculated by dividing the number of claims that can be inferred from the context by the total number of claims in the generated response.  


https://docs.ragas.io/en/v0.1.21/concepts/metrics/faithfulness.html



In [ ]:
with open(RAG_FAITHFULNESS, 'r', encoding='utf-8') as f:
        data = json.load(f) 
df_3 = pd.DataFrame(data)
df_3 = df_3[["Modified Questions", 
                "faithfulness for RAG run 1",
                "faithfulness for RAG run 2",
                "faithfulness for RAG run 3",
                "Mean faithfulness for RAG"]]
    
merged_df = pd.merge(merged_df, df_3, on="Modified Questions", how="inner")

faithfulness_mean = merged_df['Mean faithfulness for RAG'].mean()
print(f"Overall Mean Faithfulness for RAG: {faithfulness_mean:.4f}")

Overall Mean Faithfulness for RAG: 0.2678


Only 26% of retrieved statements are used. 

## RAGAS Answer Relevancy


Step 1: Reverse-engineer ‘n’ variants of the question from the generated answer using a Large Language Model (LLM)  
Step 2: Calculate the mean cosine similarity between the generated questions and the actual question.



https://docs.ragas.io/en/v0.1.21/concepts/metrics/answer_relevance.html

## Final fucntion to generate report 
(work in progress)

In [ ]:

def process_files(file_name):
    '''
    The function will output full report for the set of results for one model.

    '''
    base_name = file_name.replace('.json', '')

       # Use forward slashes or os.path.join for better cross-platform compatibility
    VANILLA_ANSWER_CORRECTNESS = f"{base_name}_vanilla_answer_correctness_evaluated.json"
    RAG_ANSWER_CORRECTNESS = f"{base_name}_rag_answer_correctness_evaluated.json"
    RAG_ANSWER_SIMILARITY = f"{base_name}_rag_answer_similarity_evaluated.json"
    VANILLA_ANSWER_SIMILARITY = f"{base_name}_vanilla_answer_similarity_evaluated.json"
    RAG_ANSWER_RELEVANCY = f"{base_name}_rag_answer_relevancy_evaluated.json"
    RAG_FAITHFULNESS = f"{base_name}_rag_faithfulness_evaluated.json"

    with open(VANILLA_ANSWER_CORRECTNESS, 'r', encoding='utf-8') as f:
        data = json.load(f)   
    df_1 = pd.DataFrame(data)

    with open(RAG_ANSWER_CORRECTNESS, 'r', encoding='utf-8') as f:
        data = json.load(f) 
    df_2 = pd.DataFrame(data)
    df_2 = df_2[["Modified Questions", 
                 "Answer Correctness for RAG run 1", 
                 "Answer Correctness for RAG run 2",
                 "Answer Correctness for RAG run 3",
                 "Mean Answer Correctness for RAG"]]
    
    merged_df = pd.merge(df_1, df_2, on="Modified Questions", how="inner")
    
    # Calculate overall means
    vanilla_mean = merged_df['Mean Answer Correctness for vanilla'].mean()
    rag_mean = merged_df['Mean Answer Correctness for RAG'].mean()
    
    print(f"Overall Mean Answer Correctness for Vanilla: {vanilla_mean:.4f}")
    print(f"Overall Mean Answer Correctness for RAG: {rag_mean:.4f}")
    print(f"Difference (RAG - Vanilla): {rag_mean - vanilla_mean:.4f}")

    with open(RAG_FAITHFULNESS, 'r', encoding='utf-8') as f:
        data = json.load(f) 
    df_3 = pd.DataFrame(data)
    df_3 = df_3[["Modified Questions", 
                "faithfulness for RAG run 1",
                "faithfulness for RAG run 2",
                "faithfulness for RAG run 3",
                "Mean faithfulness for RAG"]]
    
    merged_df = pd.merge(merged_df, df_3, on="Modified Questions", how="inner")

    faithfulness_mean = merged_df['Mean faithfulness for RAG'].mean()
    print(f"Overall Mean Faithfulness for RAG: {faithfulness_mean:.4f}")


    # Group by psychiatric category
    category_stats = merged_df.groupby('psychiatric_category').agg({
        'Mean Answer Correctness for vanilla': 'mean',
        'Mean Answer Correctness for RAG': 'mean'
    }).round(4)
    
    # Flatten column names
    category_stats.columns = ['Vanilla_Mean', 'RAG_Mean']
    
    # Add difference column
    category_stats['Difference (RAG - Vanilla)'] = (category_stats['RAG_Mean'] - 
                                                   category_stats['Vanilla_Mean']).round(4)
    
    # Sort by difference to see which categories benefit most from RAG
    category_stats = category_stats.sort_values('Difference (RAG - Vanilla)', ascending=False)
    
    print(category_stats)
    



process_files("test_dataset_together_meta-llama_Llama-4-Scout-17B-16E-Instruct_top5_answered.json")


Columns in merged dataset:
['Modified Questions', 'Reasonings', 'classification', 'psychiatry_reasoning', 'psychiatric_category', 'category_reasoning', 'category_confidence', 'Model', 'Provider', 'Generated Vanilla Answer', 'Generated RAG Answer', 'Retrieved Context', 'Top k Similarity', 'Top k Sparse', 'Sources Summary', 'Detailed Context', 'Answer Correctness for vanilla run 1', 'Answer Correctness for vanilla run 2', 'Answer Correctness for vanilla run 3', 'Mean Answer Correctness for vanilla', 'Evaluation Notes Answer Correctness for vanilla', 'Evaluation Model Answer Correctness for vanilla', 'Answer Correctness for RAG run 1', 'Answer Correctness for RAG run 2', 'Answer Correctness for RAG run 3', 'Mean Answer Correctness for RAG']

Merged dataset shape: (369, 26)
Overall Mean Answer Correctness for Vanilla: 0.6058
Overall Mean Answer Correctness for RAG: 0.5695
Difference (RAG - Vanilla): -0.0364
Overall Mean Faithfulness for RAG: 0.2678
                                         